In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import math
import scipy.stats as stats 
from numpy import array
from pandas import read_csv
from math import sqrt
from numpy import split
from plotly.subplots import make_subplots
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from datetime import datetime
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Fungsi


## preparing data

In [ ]:
def preparedata(daerah):
  # count NaN Data
  data_chl = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/chl/csv_file/'+daerah+'.csv')
  data_sst = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/sst/csv_file/'+daerah+'.csv')
  data_o2 = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/o2/csv_file/'+daerah+'.csv')
  data = pd.DataFrame(pd.Series(data_chl['time']))
  data['sst'], data['o2'], data ['chl'] =  data_sst['sst']/1000, data_o2['o2']/100, data_chl['chl']
  count_nan = data.isna().sum()
  print("Jumlah Data : ", len(data))
  print("Jumlah NaN Data : ", count_nan)
  print("Jumlah Persentase NaN Data : " ,((count_nan/len(data))*100)) 
  
  # visualize data
  # Create figure with secondary y-axis
  fig = go.Figure()
  
  # Add traces
  fig.add_trace(
      go.Scatter(x=data['time'], y=data['chl'], 
                 name="Chlorophyll (g/m3)",
                 line=dict(color='black', width=2)
                ))
  fig.update_layout(xaxis_title='<b>Date</b>',
                    font_size = 14,
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="left",
                        x=0.01,
                        font_size=10)
                    ,
                    yaxis_range=[data['chl'].min() - 0.1 ,data['chl'].max() + 0.1])
  fig.show()

  fig1 = go.Figure()
  fig1.add_trace(
      go.Scatter(x=data['time'][:869], y=data['sst'][:868], 
                 name="Sea Surface Temperature (x10^3K)",
                 line=dict(color='red', width=2)
                ))

  
  fig1.update_layout(xaxis_title='<b>Date</b>',
                    font_size = 14,
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="left",
                        x=0.01,
                        font_size=10)
                    ,
                    yaxis_range=[data['sst'][:868].min() - 0.001 ,data['sst'][:868].max() + 0.001])
  fig1.show()
  fig2 = go.Figure()
  fig2.add_trace(
      go.Scatter(x=data['time'], y=data['o2'], 
                 name="Dissolved Oxygen (mol/m3)",
                 line=dict(color='blue', width=2)
                ))

  
  fig2.update_layout(xaxis_title='<b>Date</b>',
                    font_size = 14,
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="left",
                        x=0.01,
                        font_size=10)
                    ,
                    yaxis_range=[data['o2'].min() - 0.001 ,data['o2'].max() + 0.001])
  fig2.show()
  from scipy.stats import pearsonr
  corr_sst, _ = pearsonr(data['chl'][:868],data['sst'][:868])
  corr_o2, _ = pearsonr(data['chl'],data['o2'])

  print("korelasi chl dan sst : ",corr_sst )
  print("korelasi chl dan o2 : ",corr_o2 )
  return data

## splitting data

In [ ]:
# convert data into x and y 
def change_data(train, n_input, n_out=3):
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	for _ in range(n_input, len(data)):
		in_end = in_start + n_input
		out_end = in_end + n_out
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		in_start += 1
	return array(X), array(y)

In [ ]:
def splitting_data(data):
  # end_data = list(data['time']).index(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S' ))
  data= data.loc[:,['chl']]
  data= data.reset_index(drop=True)
  
  # scaler_features = StandardScaler().fit(data_new['o2'])
  # data_new['o2'] = scaler_features.transform(data_new['o2'])

  data_new = data
  
  train = data_new[:round(len(data_new)*0.6)]
  val = data_new[round(len(data_new)*0.6) : round(len(data_new)*0.8) ]
  test = data_new[round(len(data_new)*0.8):]

  i = 0
  while len(train) % 3 != 0:
    train = data_new[:round(len(data_new)*0.7) + i]
    i += 1
  j = 0
  while len(test) % 3 != 0:
    test = data_new[round(len(data_new)*0.8) - j:]
    j += 1
  
  val = data_new[round(len(data_new)*0.7)+i : round(len(data_new)*0.8)-j]
  while len(val) % 3 != 0:
    i -= 1
    val = data_new[round(len(data_new)*0.7)+i : round(len(data_new)*0.8)-j]


  # resturvture into windows of 3 hours data
  train = array(split(train, round(len(train)/3)))
  val = array(split(val, round(len(val)/3)))
  test = array(split(test, round(len(test)/3)))

  n_input,n_out = 9,3
  n_steps_in, n_steps_out = 9, 3
  X_train, y_train =  change_data(train,n_steps_in, n_steps_out)
  X_val, y_val =  change_data(val,n_steps_in, n_steps_out)
  
  return X_train, y_train, X_val, y_val, train, val, test ,j

## model 

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True    

def define_model(unit1,unit2):
  tf.keras.backend.clear_session()
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  model1 = Sequential()
  model1.add(LSTM(unit1, activation='relu', input_shape=(n_timesteps, n_features)))
  model1.add((Dense(unit2, activation='relu')))
  model1.add((Dense(unit2, activation='relu')))
  model1.add((Dense(unit2, activation='relu')))
  model1.add((Dense(unit2, activation='relu')))
  model1.add((Dense(n_outputs)))

  # cp1 = ModelCheckpoint('model1/', save_best_only=True)
  model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), 
                 metrics=['accuracy'])
    
  return model1

def fit_model(X_train, y_train, X_val, y_val,daerah):
  # Instantiate the callback
  callbacks = myCallback()
  
  # fitting model 
  history = model1.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=100, callbacks=callbacks,verbose=2)
  # Save the trained model as a Keras HDF5 file. 
  saved_model_path = '/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/model/Heatmap_'+daerah+'.h5'
  model1.save(saved_model_path)
  return history

In [ ]:
# ini forcast buat CC
def forecast(data,daerah):

  # scaler_label = scaler_label.transform(data['chl'].values.reshape(-1, 1))
  data_new = data

  history = [x for x in val]
  predictions = list()
  n_input = 9
  for k in range(len(test)):
    datax = array(history)
    datax = datax.reshape((datax.shape[0]*datax.shape[1], datax.shape[2]))
    input_x = datax[-n_input:, :]
    input_x = input_x.reshape((1, len(input_x), 1))
    # predict 3 days 
    yhat = model1.predict(input_x, verbose=0)
    yhat_sequence = yhat[0]
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history
    history.append(test[k, :])

  prediksi = array(predictions)
  observasi = test
  observasi = test[:, :, 0]
  
  prediksi = prediksi.reshape((prediksi.shape[0]*prediksi.shape[1], 1))
  observasi = observasi.reshape((observasi.shape[0]*observasi.shape[1], 1))
  print(prediksi.shape, observasi.shape)
  test_results = pd.DataFrame(data={'Multivariate':prediksi.flatten(), 
                                   'Actuals':observasi.flatten()})
  
  test_results['Date'] = data_new['time'].iloc[round(len(data_new)*0.8) + 1 - j:].values
  test_results.to_csv('/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/chl/prediksi/'+daerah+'.csv')
  return test_results
  
def forecast_viz():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Multivariate'],
                    mode='lines',
                    name='Fitting Data Testing'))
  fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Actuals'],
                    mode='lines',
                    name='Observation'))
  
  # Edit the layout
  fig.update_layout( xaxis_title='<b>Date</b>',
                   yaxis_title='<b><i>Hs</i> [m]</b>')
  
  return fig.show()

## Evaluating Model

In [ ]:
def evaluate():
  MSE = mean_squared_error(test_results['Actuals'],test_results['Multivariate'])
  RMSE = math.sqrt(MSE)
  corr, _ = pearsonr(test_results['Actuals'],test_results['Multivariate'])

  gap_sum = 0
  for j in range(len(test_results['Actuals'])):
    gap = test_results['Multivariate'][j] - test_results['Actuals'][j]
    gap_sum += gap
    
  bias = gap_sum/len(test_results['Actuals'])
  _, pvalue = stats.ttest_ind(test_results['Actuals'],test_results['Multivariate'], equal_var=True)

  value = [round(RMSE,5),round(corr,5), round(bias,5), round(pvalue,5)]
  eval = pd.DataFrame(value, index =['rmse', 'corr', 'bias','pvalue'],
                                              columns =['values'])
  return eval

# WPP571

In [ ]:
# read data
daerah = 'WPP571'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     8
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.913242
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.5722007164542086
korelasi chl dan o2 :  0.5590352674379377


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(32,16)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 4s - loss: 0.2670 - accuracy: 0.1722 - val_loss: 0.3866 - val_accuracy: 0.1507 - 4s/epoch - 203ms/step
Epoch 2/100
19/19 - 0s - loss: 0.2641 - accuracy: 0.1722 - val_loss: 0.3830 - val_accuracy: 0.1507 - 263ms/epoch - 14ms/step
Epoch 3/100
19/19 - 0s - loss: 0.2612 - accuracy: 0.1722 - val_loss: 0.3793 - val_accuracy: 0.1507 - 255ms/epoch - 13ms/step
Epoch 4/100
19/19 - 0s - loss: 0.2581 - accuracy: 0.1722 - val_loss: 0.3753 - val_accuracy: 0.1507 - 318ms/epoch - 17ms/step
Epoch 5/100
19/19 - 0s - loss: 0.2548 - accuracy: 0.1722 - val_loss: 0.3711 - val_accuracy: 0.1507 - 263ms/epoch - 14ms/step
Epoch 6/100
19/19 - 0s - loss: 0.2513 - accuracy: 0.1722 - val_loss: 0.3667 - val_accuracy: 0.1507 - 272ms/epoch - 14ms/step
Epoch 7/100
19/19 - 0s - loss: 0.2477 - accuracy: 0.1722 - val_loss: 0.3619 - val_accuracy: 0.1507 - 244ms/epoch - 13ms/step
Epoch 8/100
19/19 - 0s - loss: 0.2437 - accuracy: 0.1722 - val_loss: 0.3568 - val_accuracy: 0.1507 - 266ms/epoch - 14ms/step
Ep

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.598062,0.859261,2022-12-01 12:00:00
1,0.603635,0.952772,2022-12-02 12:00:00
2,0.599305,0.990706,2022-12-03 12:00:00
3,0.706890,1.019436,2022-12-04 12:00:00
4,0.711513,0.905989,2022-12-05 12:00:00
...,...,...,...
172,0.459850,0.474631,2023-05-22 12:00:00
173,0.462904,0.480707,2023-05-23 12:00:00
174,0.467017,0.506096,2023-05-24 12:00:00
175,0.464921,0.537521,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.13093
corr,0.82463
bias,-0.02738
pvalue,0.17465


# WPP572

In [ ]:
# read data
daerah = 'WPP572'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     8
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.913242
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.29015109981365683
korelasi chl dan o2 :  0.08740978318768301


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 5s - loss: 0.0121 - accuracy: 0.4404 - val_loss: 0.0128 - val_accuracy: 0.4658 - 5s/epoch - 253ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0093 - accuracy: 0.4404 - val_loss: 0.0099 - val_accuracy: 0.4658 - 730ms/epoch - 38ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0066 - accuracy: 0.4404 - val_loss: 0.0068 - val_accuracy: 0.4658 - 740ms/epoch - 39ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0039 - accuracy: 0.4404 - val_loss: 0.0041 - val_accuracy: 0.4658 - 771ms/epoch - 41ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0019 - accuracy: 0.4404 - val_loss: 0.0024 - val_accuracy: 0.4658 - 746ms/epoch - 39ms/step
Epoch 6/100
19/19 - 1s - loss: 9.8211e-04 - accuracy: 0.4404 - val_loss: 0.0017 - val_accuracy: 0.4658 - 733ms/epoch - 39ms/step
Epoch 7/100
19/19 - 1s - loss: 5.4139e-04 - accuracy: 0.4404 - val_loss: 0.0013 - val_accuracy: 0.4658 - 742ms/epoch - 39ms/step
Epoch 8/100
19/19 - 1s - loss: 3.3217e-04 - accuracy: 0.4404 - val_loss: 0.0012 - val_accuracy: 0.4658 - 717ms/epoch - 

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.130837,0.132168,2022-12-01 12:00:00
1,0.131737,0.128193,2022-12-02 12:00:00
2,0.133617,0.129394,2022-12-03 12:00:00
3,0.125066,0.127918,2022-12-04 12:00:00
4,0.125773,0.130326,2022-12-05 12:00:00
...,...,...,...
172,0.100183,0.099010,2023-05-22 12:00:00
173,0.098063,0.097700,2023-05-23 12:00:00
174,0.102472,0.094768,2023-05-24 12:00:00
175,0.102008,0.094923,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.04851
corr,0.66501
bias,-0.00859
pvalue,0.08350


# WPP573

In [ ]:
# read data
daerah = 'WPP573'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     8
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.913242
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.6058760780060273
korelasi chl dan o2 :  0.7622311594368112


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 4s - loss: 0.0875 - accuracy: 0.1424 - val_loss: 0.0223 - val_accuracy: 0.0959 - 4s/epoch - 223ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0814 - accuracy: 0.1424 - val_loss: 0.0184 - val_accuracy: 0.0959 - 740ms/epoch - 39ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0742 - accuracy: 0.1424 - val_loss: 0.0142 - val_accuracy: 0.0959 - 717ms/epoch - 38ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0656 - accuracy: 0.1424 - val_loss: 0.0094 - val_accuracy: 0.0959 - 760ms/epoch - 40ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0541 - accuracy: 0.1424 - val_loss: 0.0046 - val_accuracy: 0.0959 - 761ms/epoch - 40ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0406 - accuracy: 0.1424 - val_loss: 0.0045 - val_accuracy: 0.0959 - 704ms/epoch - 37ms/step
Epoch 7/100
19/19 - 1s - loss: 0.0327 - accuracy: 0.1424 - val_loss: 0.0055 - val_accuracy: 0.0959 - 716ms/epoch - 38ms/step
Epoch 8/100
19/19 - 1s - loss: 0.0290 - accuracy: 0.1424 - val_loss: 0.0040 - val_accuracy: 0.0959 - 735ms/epoch - 39ms/step
Ep

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.132843,0.109376,2022-12-01 12:00:00
1,0.131983,0.110622,2022-12-02 12:00:00
2,0.147470,0.110943,2022-12-03 12:00:00
3,0.115970,0.110865,2022-12-04 12:00:00
4,0.119663,0.110562,2022-12-05 12:00:00
...,...,...,...
172,0.234589,0.156401,2023-05-22 12:00:00
173,0.238613,0.154167,2023-05-23 12:00:00
174,0.191140,0.151837,2023-05-24 12:00:00
175,0.186886,0.151006,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.05708
corr,0.78561
bias,0.01654
pvalue,0.06237


# WPP573

In [ ]:
# read data
daerah = 'WPP573'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     8
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.913242
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.6058760780060273
korelasi chl dan o2 :  0.7622311594368112


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 3s - loss: 0.0883 - accuracy: 0.1507 - val_loss: 0.0235 - val_accuracy: 0.0959 - 3s/epoch - 176ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0850 - accuracy: 0.1424 - val_loss: 0.0214 - val_accuracy: 0.0959 - 699ms/epoch - 37ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0809 - accuracy: 0.1424 - val_loss: 0.0188 - val_accuracy: 0.0959 - 687ms/epoch - 36ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0755 - accuracy: 0.1424 - val_loss: 0.0155 - val_accuracy: 0.0959 - 706ms/epoch - 37ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0675 - accuracy: 0.1424 - val_loss: 0.0107 - val_accuracy: 0.0959 - 671ms/epoch - 35ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0537 - accuracy: 0.1424 - val_loss: 0.0046 - val_accuracy: 0.0959 - 892ms/epoch - 47ms/step
Epoch 7/100
19/19 - 1s - loss: 0.0360 - accuracy: 0.1424 - val_loss: 0.0058 - val_accuracy: 0.0959 - 1s/epoch - 56ms/step
Epoch 8/100
19/19 - 1s - loss: 0.0298 - accuracy: 0.1424 - val_loss: 0.0028 - val_accuracy: 0.0959 - 1s/epoch - 58ms/step
Epoch 9/

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.139562,0.109376,2022-12-01 12:00:00
1,0.143141,0.110622,2022-12-02 12:00:00
2,0.150440,0.110943,2022-12-03 12:00:00
3,0.119667,0.110865,2022-12-04 12:00:00
4,0.126297,0.110562,2022-12-05 12:00:00
...,...,...,...
172,0.259890,0.156401,2023-05-22 12:00:00
173,0.255106,0.154167,2023-05-23 12:00:00
174,0.204737,0.151837,2023-05-24 12:00:00
175,0.204796,0.151006,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.06504
corr,0.74757
bias,0.02178
pvalue,0.01711


# WPP711

In [ ]:
# read data
daerah = 'WPP711'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     8
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.913242
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  0.1050762263572831
korelasi chl dan o2 :  0.6379789404870513


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 6s - loss: 0.1081 - accuracy: 0.1589 - val_loss: 0.0534 - val_accuracy: 0.1507 - 6s/epoch - 304ms/step
Epoch 2/100
19/19 - 1s - loss: 0.1033 - accuracy: 0.1589 - val_loss: 0.0499 - val_accuracy: 0.1507 - 908ms/epoch - 48ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0973 - accuracy: 0.1589 - val_loss: 0.0451 - val_accuracy: 0.1507 - 733ms/epoch - 39ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0890 - accuracy: 0.1589 - val_loss: 0.0383 - val_accuracy: 0.1507 - 883ms/epoch - 46ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0757 - accuracy: 0.1589 - val_loss: 0.0268 - val_accuracy: 0.1507 - 1s/epoch - 58ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0509 - accuracy: 0.1589 - val_loss: 0.0078 - val_accuracy: 0.1507 - 1s/epoch - 61ms/step
Epoch 7/100
19/19 - 1s - loss: 0.0218 - accuracy: 0.1589 - val_loss: 0.0035 - val_accuracy: 0.1507 - 1s/epoch - 60ms/step
Epoch 8/100
19/19 - 1s - loss: 0.0176 - accuracy: 0.3245 - val_loss: 0.0013 - val_accuracy: 0.1507 - 970ms/epoch - 51ms/step
Epoch 9/100

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.277909,0.182503,2022-12-01 12:00:00
1,0.282460,0.180817,2022-12-02 12:00:00
2,0.280907,0.187667,2022-12-03 12:00:00
3,0.258234,0.195653,2022-12-04 12:00:00
4,0.272848,0.194603,2022-12-05 12:00:00
...,...,...,...
172,0.385122,0.326078,2023-05-22 12:00:00
173,0.385920,0.371255,2023-05-23 12:00:00
174,0.324341,0.411674,2023-05-24 12:00:00
175,0.354384,0.442926,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.14819
corr,0.46499
bias,-0.01795
pvalue,0.17524


# WPP712

In [ ]:
# read data
daerah = 'WPP712'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     8
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.913242
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.527121040571012
korelasi chl dan o2 :  0.14328320256422694


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 4s - loss: 0.0816 - accuracy: 0.4834 - val_loss: 0.0324 - val_accuracy: 0.4110 - 4s/epoch - 227ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0775 - accuracy: 0.4818 - val_loss: 0.0293 - val_accuracy: 0.4110 - 683ms/epoch - 36ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0726 - accuracy: 0.4818 - val_loss: 0.0256 - val_accuracy: 0.4110 - 678ms/epoch - 36ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0659 - accuracy: 0.4818 - val_loss: 0.0210 - val_accuracy: 0.4110 - 659ms/epoch - 35ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0563 - accuracy: 0.4818 - val_loss: 0.0149 - val_accuracy: 0.4110 - 689ms/epoch - 36ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0421 - accuracy: 0.4818 - val_loss: 0.0093 - val_accuracy: 0.4110 - 678ms/epoch - 36ms/step
Epoch 7/100
19/19 - 1s - loss: 0.0301 - accuracy: 0.4818 - val_loss: 0.0090 - val_accuracy: 0.4110 - 693ms/epoch - 36ms/step
Epoch 8/100
19/19 - 1s - loss: 0.0245 - accuracy: 0.4818 - val_loss: 0.0062 - val_accuracy: 0.4110 - 675ms/epoch - 36ms/step
Ep

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.166863,0.171086,2022-12-01 12:00:00
1,0.175096,0.168551,2022-12-02 12:00:00
2,0.176139,0.170735,2022-12-03 12:00:00
3,0.169068,0.178173,2022-12-04 12:00:00
4,0.177201,0.179809,2022-12-05 12:00:00
...,...,...,...
172,0.148854,0.124992,2023-05-22 12:00:00
173,0.153677,0.124494,2023-05-23 12:00:00
174,0.131554,0.125209,2023-05-24 12:00:00
175,0.141029,0.127610,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.13588
corr,0.83315
bias,-0.01502
pvalue,0.52838


# WPP713

In [ ]:
# read data
daerah = 'WPP713'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     8
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.913242
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.4018953566635223
korelasi chl dan o2 :  0.28743022415681946


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 3s - loss: 0.0272 - accuracy: 0.3974 - val_loss: 0.0179 - val_accuracy: 0.5479 - 3s/epoch - 182ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0258 - accuracy: 0.3974 - val_loss: 0.0167 - val_accuracy: 0.5479 - 702ms/epoch - 37ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0244 - accuracy: 0.3974 - val_loss: 0.0154 - val_accuracy: 0.5479 - 741ms/epoch - 39ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0228 - accuracy: 0.3974 - val_loss: 0.0139 - val_accuracy: 0.5479 - 711ms/epoch - 37ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0208 - accuracy: 0.3974 - val_loss: 0.0121 - val_accuracy: 0.5479 - 859ms/epoch - 45ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0182 - accuracy: 0.3974 - val_loss: 0.0096 - val_accuracy: 0.5479 - 1s/epoch - 60ms/step
Epoch 7/100
19/19 - 1s - loss: 0.0145 - accuracy: 0.3974 - val_loss: 0.0069 - val_accuracy: 0.5479 - 1s/epoch - 60ms/step
Epoch 8/100
19/19 - 1s - loss: 0.0109 - accuracy: 0.3974 - val_loss: 0.0044 - val_accuracy: 0.5479 - 1s/epoch - 58ms/step
Epoch 9/100

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.157399,0.143061,2022-12-01 12:00:00
1,0.156231,0.144913,2022-12-02 12:00:00
2,0.160848,0.148004,2022-12-03 12:00:00
3,0.151734,0.149395,2022-12-04 12:00:00
4,0.151123,0.150350,2022-12-05 12:00:00
...,...,...,...
172,0.129707,0.119972,2023-05-22 12:00:00
173,0.128594,0.119811,2023-05-23 12:00:00
174,0.124633,0.118894,2023-05-24 12:00:00
175,0.126921,0.118148,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.06565
corr,0.56772
bias,-0.00233
pvalue,0.74972


# WPP714

In [ ]:
# read data
daerah = 'WPP714'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     7
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.799087
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.42053747029363064
korelasi chl dan o2 :  0.5675954716249993


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 3s - loss: 0.0206 - accuracy: 0.3990 - val_loss: 0.0153 - val_accuracy: 0.2877 - 3s/epoch - 176ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0197 - accuracy: 0.3990 - val_loss: 0.0145 - val_accuracy: 0.2877 - 725ms/epoch - 38ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0189 - accuracy: 0.3990 - val_loss: 0.0138 - val_accuracy: 0.2877 - 693ms/epoch - 36ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0181 - accuracy: 0.3990 - val_loss: 0.0131 - val_accuracy: 0.2877 - 968ms/epoch - 51ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0173 - accuracy: 0.3990 - val_loss: 0.0124 - val_accuracy: 0.2877 - 1s/epoch - 57ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0165 - accuracy: 0.3990 - val_loss: 0.0118 - val_accuracy: 0.2877 - 1s/epoch - 55ms/step
Epoch 7/100
19/19 - 1s - loss: 0.0159 - accuracy: 0.3990 - val_loss: 0.0114 - val_accuracy: 0.2877 - 1s/epoch - 55ms/step
Epoch 8/100
19/19 - 1s - loss: 0.0154 - accuracy: 0.3990 - val_loss: 0.0109 - val_accuracy: 0.2877 - 931ms/epoch - 49ms/step
Epoch 9/100

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.119254,0.105664,2022-12-01 12:00:00
1,0.117999,0.107985,2022-12-02 12:00:00
2,0.116746,0.110678,2022-12-03 12:00:00
3,0.118040,0.113314,2022-12-04 12:00:00
4,0.115540,0.117261,2022-12-05 12:00:00
...,...,...,...
172,0.111725,0.101680,2023-05-22 12:00:00
173,0.111174,0.100565,2023-05-23 12:00:00
174,0.115942,0.099821,2023-05-24 12:00:00
175,0.111022,0.098982,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.02313
corr,0.42912
bias,0.00397
pvalue,0.08148


# WPP715

In [ ]:
# read data
daerah = 'WPP715'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     7
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.799087
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.03670679766932221
korelasi chl dan o2 :  0.26568162978843374


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 5s - loss: 0.0134 - accuracy: 0.4437 - val_loss: 0.0102 - val_accuracy: 0.5205 - 5s/epoch - 285ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0100 - accuracy: 0.4437 - val_loss: 0.0072 - val_accuracy: 0.5205 - 1s/epoch - 69ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0070 - accuracy: 0.4437 - val_loss: 0.0047 - val_accuracy: 0.5205 - 1s/epoch - 65ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0045 - accuracy: 0.4437 - val_loss: 0.0030 - val_accuracy: 0.5205 - 934ms/epoch - 49ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0031 - accuracy: 0.4437 - val_loss: 0.0022 - val_accuracy: 0.5205 - 800ms/epoch - 42ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0024 - accuracy: 0.4437 - val_loss: 0.0017 - val_accuracy: 0.5205 - 801ms/epoch - 42ms/step
Epoch 7/100
19/19 - 1s - loss: 0.0018 - accuracy: 0.4437 - val_loss: 0.0012 - val_accuracy: 0.5205 - 819ms/epoch - 43ms/step
Epoch 8/100
19/19 - 1s - loss: 0.0014 - accuracy: 0.4437 - val_loss: 9.0308e-04 - val_accuracy: 0.5205 - 815ms/epoch - 43ms/step
Epoc

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.123627,0.116789,2022-12-01 12:00:00
1,0.123403,0.118571,2022-12-02 12:00:00
2,0.123657,0.120952,2022-12-03 12:00:00
3,0.123059,0.119086,2022-12-04 12:00:00
4,0.122724,0.118322,2022-12-05 12:00:00
...,...,...,...
172,0.123503,0.137060,2023-05-22 12:00:00
173,0.123815,0.147282,2023-05-23 12:00:00
174,0.125877,0.146687,2023-05-24 12:00:00
175,0.126020,0.144394,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.00642
corr,0.59655
bias,0.00237
pvalue,0.00003


# WPP716

In [ ]:
# read data
daerah = 'WPP716'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     7
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.799087
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.2682984271524104
korelasi chl dan o2 :  -0.056555698715031945


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 4s - loss: 0.0155 - accuracy: 0.1225 - val_loss: 0.0130 - val_accuracy: 0.1096 - 4s/epoch - 195ms/step
Epoch 2/100
19/19 - 1s - loss: 0.0125 - accuracy: 0.1225 - val_loss: 0.0100 - val_accuracy: 0.1096 - 753ms/epoch - 40ms/step
Epoch 3/100
19/19 - 1s - loss: 0.0094 - accuracy: 0.1225 - val_loss: 0.0071 - val_accuracy: 0.1096 - 741ms/epoch - 39ms/step
Epoch 4/100
19/19 - 1s - loss: 0.0066 - accuracy: 0.1225 - val_loss: 0.0045 - val_accuracy: 0.1096 - 690ms/epoch - 36ms/step
Epoch 5/100
19/19 - 1s - loss: 0.0038 - accuracy: 0.1225 - val_loss: 0.0022 - val_accuracy: 0.1096 - 718ms/epoch - 38ms/step
Epoch 6/100
19/19 - 1s - loss: 0.0016 - accuracy: 0.1225 - val_loss: 6.3539e-04 - val_accuracy: 0.1096 - 1s/epoch - 64ms/step
Epoch 7/100
19/19 - 1s - loss: 4.7143e-04 - accuracy: 0.1225 - val_loss: 1.5983e-04 - val_accuracy: 0.1096 - 1s/epoch - 65ms/step
Epoch 8/100
19/19 - 1s - loss: 1.6355e-04 - accuracy: 0.1225 - val_loss: 6.8305e-05 - val_accuracy: 0.1096 - 1s/epoch - 7

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.135019,0.129887,2022-12-01 12:00:00
1,0.134735,0.131019,2022-12-02 12:00:00
2,0.134270,0.132664,2022-12-03 12:00:00
3,0.134472,0.134176,2022-12-04 12:00:00
4,0.134184,0.134492,2022-12-05 12:00:00
...,...,...,...
172,0.129797,0.128567,2023-05-22 12:00:00
173,0.129673,0.129666,2023-05-23 12:00:00
174,0.128041,0.130852,2023-05-24 12:00:00
175,0.127818,0.130567,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.00459
corr,0.75123
bias,-0.00020
pvalue,0.75743


# WPP717

In [ ]:
# read data
daerah = 'WPP717'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     7
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.799087
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.057467633996052016
korelasi chl dan o2 :  -0.12598535345653775


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(512,20)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 6s - loss: 0.0570 - accuracy: 0.4437 - val_loss: 0.0508 - val_accuracy: 0.4795 - 6s/epoch - 317ms/step
Epoch 2/100
19/19 - 3s - loss: 0.0487 - accuracy: 0.4437 - val_loss: 0.0406 - val_accuracy: 0.4795 - 3s/epoch - 181ms/step
Epoch 3/100
19/19 - 4s - loss: 0.0344 - accuracy: 0.4437 - val_loss: 0.0205 - val_accuracy: 0.4795 - 4s/epoch - 209ms/step
Epoch 4/100
19/19 - 3s - loss: 0.0110 - accuracy: 0.4437 - val_loss: 0.0025 - val_accuracy: 0.4795 - 3s/epoch - 157ms/step
Epoch 5/100
19/19 - 3s - loss: 0.0027 - accuracy: 0.3212 - val_loss: 7.4179e-04 - val_accuracy: 0.4110 - 3s/epoch - 152ms/step
Epoch 6/100
19/19 - 3s - loss: 0.0021 - accuracy: 0.3659 - val_loss: 6.2178e-04 - val_accuracy: 0.1096 - 3s/epoch - 152ms/step
Epoch 7/100
19/19 - 4s - loss: 0.0020 - accuracy: 0.3146 - val_loss: 5.9119e-04 - val_accuracy: 0.4795 - 4s/epoch - 227ms/step
Epoch 8/100
19/19 - 3s - loss: 0.0020 - accuracy: 0.3742 - val_loss: 5.8207e-04 - val_accuracy: 0.1233 - 3s/epoch - 154ms/step


In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.232624,0.247283,2022-12-01 12:00:00
1,0.232020,0.257420,2022-12-02 12:00:00
2,0.233434,0.265979,2022-12-03 12:00:00
3,0.249643,0.265649,2022-12-04 12:00:00
4,0.246104,0.270935,2022-12-05 12:00:00
...,...,...,...
172,0.239181,0.242139,2023-05-22 12:00:00
173,0.240971,0.239811,2023-05-23 12:00:00
174,0.244640,0.236801,2023-05-24 12:00:00
175,0.244516,0.229268,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.03178
corr,0.51687
bias,0.01064
pvalue,0.00030


# WPP718

In [ ]:
# read data
daerah = 'WPP718'
data = preparedata(daerah)

Jumlah Data :  876
Jumlah NaN Data :  time    0
sst     7
o2      0
chl     0
dtype: int64
Jumlah Persentase NaN Data :  time    0.000000
sst     0.799087
o2      0.000000
chl     0.000000
dtype: float64


korelasi chl dan sst :  -0.8203747340466464
korelasi chl dan o2 :  0.64262033455585


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((604, 9, 1), (604, 3), (73, 9, 1), (73, 3))

In [ ]:
train.shape, val.shape, test.shape

((205, 3, 1), (28, 3, 1), (59, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(96,16)
history = fit_model(X_train, y_train, X_val, y_val,daerah)

Epoch 1/100
19/19 - 5s - loss: 1.0465 - accuracy: 0.3146 - val_loss: 0.2606 - val_accuracy: 0.5616 - 5s/epoch - 246ms/step
Epoch 2/100
19/19 - 0s - loss: 0.9975 - accuracy: 0.4785 - val_loss: 0.2267 - val_accuracy: 0.5616 - 264ms/epoch - 14ms/step
Epoch 3/100
19/19 - 0s - loss: 0.9348 - accuracy: 0.4768 - val_loss: 0.1785 - val_accuracy: 0.5616 - 256ms/epoch - 13ms/step
Epoch 4/100
19/19 - 0s - loss: 0.7274 - accuracy: 0.4404 - val_loss: 0.0989 - val_accuracy: 0.5616 - 259ms/epoch - 14ms/step
Epoch 5/100
19/19 - 0s - loss: 0.4396 - accuracy: 0.4785 - val_loss: 0.0337 - val_accuracy: 0.5616 - 244ms/epoch - 13ms/step
Epoch 6/100
19/19 - 0s - loss: 0.2822 - accuracy: 0.4768 - val_loss: 0.0124 - val_accuracy: 0.5616 - 268ms/epoch - 14ms/step
Epoch 7/100
19/19 - 0s - loss: 0.2386 - accuracy: 0.4288 - val_loss: 0.0154 - val_accuracy: 0.5616 - 244ms/epoch - 13ms/step
Epoch 8/100
19/19 - 0s - loss: 0.2217 - accuracy: 0.4172 - val_loss: 0.0116 - val_accuracy: 0.3288 - 275ms/epoch - 14ms/step
Ep

In [ ]:
test_results = forecast(data,daerah)
test_results

(177, 1) (177, 1)


,Multivariate,Actuals,Date
0,0.309709,0.267877,2022-12-01 12:00:00
1,0.301425,0.267851,2022-12-02 12:00:00
2,0.304940,0.262296,2022-12-03 12:00:00
3,0.297361,0.260281,2022-12-04 12:00:00
4,0.284925,0.259966,2022-12-05 12:00:00
...,...,...,...
172,0.416952,0.840293,2023-05-22 12:00:00
173,0.398541,1.177471,2023-05-23 12:00:00
174,1.104867,1.292007,2023-05-24 12:00:00
175,1.113739,1.144733,2023-05-25 12:00:00


In [ ]:
forecast_viz()

In [ ]:
error= evaluate()
error

,values
rmse,0.11818
corr,0.78745
bias,0.01001
pvalue,0.60076
